In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 60.7 MB/s 
     |████████████████████████████████| 596 kB 76.2 MB/s 
     |████████████████████████████████| 6.6 MB 47.9 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
!pip install hanja

     |████████████████████████████████| 120 kB 4.3 MB/s 
     |████████████████████████████████| 265 kB 15.2 MB/s 
     |████████████████████████████████| 207 kB 50.1 MB/s 
     |████████████████████████████████| 297 kB 3.5 MB/s 
     |████████████████████████████████| 207 kB 70.2 MB/s 
     |████████████████████████████████| 207 kB 75.6 MB/s 
     |████████████████████████████████| 213 kB 72.0 MB/s 
     |████████████████████████████████| 213 kB 69.6 MB/s 
     |████████████████████████████████| 213 kB 73.5 MB/s 
     |████████████████████████████████| 213 kB 74.6 MB/s 
     |████████████████████████████████| 253 kB 71.3 MB/s 
     |████████████████████████████████| 252 kB 65.2 MB/s 
     |████████████████████████████████| 252 kB 68.5 MB/s 
     |████████████████████████████████| 242 kB 64.1 MB/s 
     |████████████████████████████████| 242 kB 64.8 MB/s 
     |████████████████████████████████| 242 kB 70.3 MB/s 
     |████████████████████████████████| 229 kB 71.8 MB/s 
     |██████████

In [7]:
from transformers import BertTokenizerFast, AlbertModel, BertModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import hanja
from imblearn.over_sampling import RandomOverSampler
from transformers.optimization import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup
from transformers import AdamW
from collections import defaultdict
import torchvision.models as models
import os

In [8]:
from google.colab import drive
drive.mount('/content/drive')
os.listdir("/content/drive/My Drive/3조")

Mounted at /content/drive


['1.기획안&WBS',
 '2.수행일지',
 '3.최종포트폴리오',
 '4.과제',
 '멘토링',
 '회의록',
 '[D11&12]기업요구사항 기반의 문제해결 프로젝트.pdf',
 'Data',
 '빅데이터분석프로젝트_단계별공정.pdf',
 'DS_folder',
 'DE']

In [9]:
PATH = '/content/drive/My Drive/3조/DS_folder/jupyter notebook/Data/'

In [76]:
train = pd.read_csv(PATH + 'data.csv', index_col=0)
test = pd.read_csv(PATH + 'val.csv', index_col=0)

In [77]:
train

,target,data
0,0,지금 난 기분이 너무 좋아.
1,0,나도 조카가 생겨! 너무 기뻐.
2,1,거래처와의 다음 계약이 무산될까봐 불안해.
3,2,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...
4,2,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.
...,...,...
199367,2,일단 좀 쉬어야 할 것 같아. 남자 친구와의 추억도 정리하고 마음도 정리해야겠어.
199368,2,나는 아들이 엄청나게 좋아할 거라 기대했는데 실망스러워.
199369,5,먼저 업무 인계서부터 천천히 작성 해야지.
199370,0,기분이 좋아. 나도 일 때문에 바쁠 때는 미리 얘기하기로 했어.


In [85]:
x_train = train['data']
y_train = train['target']
x_test = test['data']
y_test = test['target']

---

In [86]:
tokenizer_bert_kor_base = BertTokenizerFast.from_pretrained("kykim/albert-kor-base")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizerFast'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizerFast'. 
The class this function is called from is 'BertTokenizerFast'.


In [87]:
titles_t = x_train.to_numpy().reshape(-1,1)
labels_t = y_train.to_numpy().reshape(-1,1)
# oversample = RandomOverSampler()
# X_over, y_over = oversample.fit_resample(titles_t, labels_t)
# train = pd.DataFrame({'title':X_over.reshape(-1), 'topic_idx':y_over.reshape(-1)})
train = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

In [88]:
titles_t = x_test.to_numpy().reshape(-1,1)
labels_t = y_test.to_numpy().reshape(-1,1)
# oversample = RandomOverSampler()
# X_over, y_over = oversample.fit_resample(titles_t, labels_t)
# train = pd.DataFrame({'title':X_over.reshape(-1), 'topic_idx':y_over.reshape(-1)})
test = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

In [89]:
# 뉴스 데이터 클래스 생성
class NewsSubjectDataset(Dataset):
  def __init__(self, subjects, targets, tokenizer, max_len):
    self.subjects = subjects
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  def __len__(self):
    return len(self.subjects)
  def __getitem__(self, item):
    subject = str(self.subjects[item])
    target = self.targets[item]
    encoding = self.tokenizer.encode_plus(
      subject,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      padding = 'max_length',
      truncation = True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    return {
      'subject_text': subject,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }
def create_data_loader(df, tokenizer, max_len, batch_size, shuffle_=False, valid=False):
  if valid:
    ds = NewsSubjectDataset(
      subjects=df.title.to_numpy(),
      targets=np.zeros(len(df)),
      tokenizer=tokenizer,
      max_len=max_len
      )
  else:
    ds = NewsSubjectDataset(
      subjects=df.title.to_numpy(),
      targets=df.topic_idx.to_numpy(),
      tokenizer=tokenizer,
      max_len=max_len
    )
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4,
    shuffle = shuffle_
  )
  
# 데이터 로더 생성
BATCH_SIZE =64
MAX_LEN =32
train_data_loader = create_data_loader(train, tokenizer_bert_kor_base, MAX_LEN, BATCH_SIZE, shuffle_=True)
valid_data_loader = create_data_loader(test, tokenizer_bert_kor_base, MAX_LEN, BATCH_SIZE, valid=True)
import random
class NewsSubjectClassifier(nn.Module):
  def __init__(self, n_classes):
    super(NewsSubjectClassifier, self).__init__()
    self.bert = AlbertModel.from_pretrained("kykim/albert-kor-base")
    self.drop = nn.Dropout(p=0.5)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
       return_dict=False
    )
    output = self.drop(pooled_output)
    return self.out(output)

device = torch.device("cpu")
def get_predictions(model, data_loader):
  model = model.eval()
  subject_texts = []
  predictions = []
  prediction_probs = []
  with torch.no_grad():
    for d in data_loader:
      texts = d["subject_text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      subject_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  return subject_texts, predictions, prediction_probs
import gc
EPOCHS = 2

In [21]:
model_albert_kor_base = NewsSubjectClassifier(n_classes=6).to(device)
optimizer = AdamW(model_albert_kor_base.parameters(), lr=1e-4)
total_steps = len(train_data_loader) * EPOCHS
## cosine_scheduler or linear,  warmup or no warmup
scheduler = get_cosine_schedule_with_warmup(
  optimizer,
  num_warmup_steps=int(total_steps*0.1),
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

from tqdm import tqdm
def train_epoch(model,data_loader,loss_fn,optimizer,device,scheduler,n_examples):
  model = model.train()
  losses = []
  correct_predictions = 0
  for d in tqdm(data_loader):
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)
    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)
    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double() / n_examples, np.mean(losses)

Downloading:   0%|          | 0.00/50.9M [00:00<?, ?B/s]

Some weights of the model checkpoint at kykim/albert-kor-base were not used when initializing AlbertModel: ['sop_classifier.classifier.bias', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.bias', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)
  train_acc, train_loss = train_epoch(
    model_albert_kor_base,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(train)
  )
  print(f'Train loss {train_loss} accuracy {train_acc}')

y_review_texts, y_pred, y_pred_probs = get_predictions(
  model_albert_kor_base,
  valid_data_loader
)
answers = []
answers.append(y_pred.tolist())
gc.collect()
torch.cuda.empty_cache()

Epoch 1/2
----------


100%|██████████| 3116/3116 [4:53:15<00:00,  5.65s/it]


Train loss 1.480560501269872 accuracy 0.43501595008326144
Epoch 2/2
----------


100%|██████████| 3116/3116 [4:48:34<00:00,  5.56s/it]

Train loss 1.2282276372969227 accuracy 0.5251589992576691


In [23]:
# torch.save(model_albert_kor_base, PATH+'model_albert_kor_base.pth')

In [27]:
# torch.save({
#             'epoch': epoch,
#             'model_state_dict': model_albert_kor_base.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'loss': train_loss,
#             }, './모델.tar')

In [90]:
model = torch.load(PATH+"model_albert_kor_base.pth")

In [91]:
state_dict = model.load_state_dict(torch.load(PATH + 'model_state.pth'))

In [92]:
model.eval()

NewsSubjectClassifier(
  (bert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(42000, 128, padding_idx=0)
      (position_embeddings): Embedding(256, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)


In [93]:
subject_texts, predictions, prediction_probs = get_predictions(model, valid_data_loader)

In [104]:
df = pd.DataFrame({'data':subject_texts, 'predict':predictions, 'target':y_test})
df

,data,predict,target
0,요즘 젊은 직원들 대하기가 여간 조심스러운게 아니야.,1,1
1,요즘 이 책을 읽는데 너무 괜찮은 책 인 것 같아. 마음이 평온해져.,0,0
2,치료를 계속 받아야 하는 친구가 있는데 병원비 때문에 퇴원한다는 거 내가 돈을 빌려줬어.,4,0
3,남편이 공들여 요리를 해주는 날은 사랑받는 기분이 들어. 그게 바로 오늘이고!,0,0
4,어제까지 나와 제일 친했던 친구가 오늘은 인사도 안해.,2,1
...,...,...,...
24918,모르는 건 나 자신과 회사를 위해서 물론 배워야겠지. 내일은 그 동료에게 도움을 부...,4,2
24919,각자의 시간을 가져봐야겠어.,2,4
24920,잔소리만 하는 게 아니야. 옆에 있는 것도 이제 지겨워.,5,3
24921,그래야 할거 같아 내가 배우고 싶은걸 생각해봐야겠어.,2,4


In [115]:
# 정확도
df[df['predict'] == df['target']].shape[0] / df.shape[0]

0.5171929542992416

In [ ]:
df['data']

In [ ]:
# def change(x):
#     if x == '기쁨':
#         return 0
#     elif x == '불안':
#         return 1
#     elif x == '슬픔':
#         return 2
#     elif x ==‘당황:
#         return 3
#     elif x == ‘상처’:
#         return 4
#     elif x == ‘분노’:
#         return 5